In [36]:
from neo4j.v1 import GraphDatabase, basic_auth
import itertools
from joblib import Parallel, delayed
import multiprocessing
import time

In [37]:
num_cores = multiprocessing.cpu_count()

In [38]:
driver = GraphDatabase.driver("bolt://localhost", auth=basic_auth("neo4j", "dmkm1234"))

In [39]:
session = driver.session()

In [42]:
t = time.time()
for i in range(0,400):
    session.run("MATCH (ar:Articles)-[:isArticle]-(s:Signatures)-[:isAuthor]-(au:Authors) WHERE ar.article_id in range (%s,%s) WITH (ar.article_id) as article, collect(au.author_id) as authorlist MATCH (au1:Authors),(au2:Authors) WHERE au1.author_id in authorlist and au2.author_id in authorlist MERGE (au1)-[r:isCoauthor]-(au2)" % (i*1000,(i+1)*1000))
elapsed = time.time() - t
print(elapsed)

0.119999885559


In [ ]:
session.close()

In [13]:
from math import sqrt
[sqrt(i ** 2) for i in range(10)]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]